In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
class node(object):
    
    def __init__(self,state,inbound=None,outbound=None):
        self.state=state
        self.inbound={}
        self.outbound={}
        
    def __str__(self):
        string="State: "+str(self.state)
        return string
    
    def addInbound(self,comingFrom,arcCost):
        self.inbound[comingFrom]=arcCost
        
    def addOutbound(self,goingTo,arcCost):
        self.outbound[goingTo]=arcCost


In [3]:
n=6
U=10

np.random.seed(78440)
    
cvals=np.random.randint(1,U,n)
wvals=np.random.randint(1,U,n)
W=sum(wvals)/2
d=[0,1]

In [4]:
n1=50
U1=50
    
cvals1=np.random.randint(1,U1,n1)
wvals1=np.random.randint(1,U1,n1)
W1=sum(wvals1)/2
d1=[0,1]

def read_instance(address = 'instance.csv'):
    data = pd.read_csv(address)
    c = data['c'].values
    w = data['w'].values
    W = data['W'].values[0]
    return c,w,W

In [5]:
ntest=4

cvalstest=[7,8,6,14]
wvalstest=[3,3,4,6]
Wtest=6

In [6]:
def Algorithm_1(n,cvals,wvals,W,d):
    
    root_node=node(0)
    all_layers={}
    all_layers[0]={}
    all_layers[0][root_node.state]=root_node
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]={}
        nextLayer=all_layers[x]
        #Iterate through each node and action decision
        for currW, nodeInstance in currLayer.items():
            for decision in d:
                newstate=currW+(wvals[x-1]*decision)
                #Feasability check
                #Check if the new state already exists in the layer
                if newstate<=W:
                    if newstate in nextLayer:
                        #Add arc into arclist
                        cost=cvals[x-1]*decision
                        #Add arc into node inbound and outbound
                        nodeInstance.outbound[nextLayer[newstate]]=cost
                        nextLayer[newstate].inbound[nodeInstance]=cost
                    else:
                        #Create new node, add arc to arclist
                        new_node=node(newstate)
                        nextLayer[newstate]=new_node
                        cost=cvals[x-1]*decision
                        #Add arc to node inbound and outbound
                        nodeInstance.outbound[new_node]=cost
                        new_node.inbound[nodeInstance]=cost
        
    #Changed from W because Rachel said so
    terminal_node=node(-999)
    all_layers[n]={}
    all_layers[n][-999]=terminal_node
    
    for currW, nodeInstance in all_layers[n-1].items():
        for decision in d:
            newstate=currW+(wvals[n-1]*decision)
            if newstate<=W:
                cost=cvals[n-1]*decision
                nodeInstance.outbound[terminal_node]=cost
                terminal_node.inbound[nodeInstance]=cost
                
    #final_node=node(0)            
    #terminal_node.outbound[final_node]=0
                
    return all_layers

In [7]:
def Algorithm_2(all_layers,n):
     #Final reduction algorithm here we go
    reduced_layers=all_layers.copy()
    
    #Starting from the second last layer moving up
    for x in range(n-1,0,-1):
        #all_tails_found=False
        to_remove=[]
        tail_list={}
        for nodeInstance in reduced_layers[x].values():
            #Add ending combination if not already in the tail list
            if tuple(nodeInstance.outbound.items()) not in tail_list:
                tail_list[tuple(nodeInstance.outbound.items())]=nodeInstance
            
            #If ending combination exists
            else:
                to_remove.append(nodeInstance)
                #Redirect arcs
                for comingFrom, arcCost in nodeInstance.inbound.items():
                    tail_list[tuple(nodeInstance.outbound.items())].inbound[comingFrom]=arcCost
                    comingFrom.outbound[tail_list[tuple(nodeInstance.outbound.items())]]=arcCost
                    #Remove arcs from comingFrom node
                    if nodeInstance in comingFrom.outbound:
                        del(comingFrom.outbound[nodeInstance])
                        
        #Remove all duplicate nodes            
        for remove_node in to_remove:
            del(reduced_layers[x][remove_node.state])
        
    return reduced_layers

In [8]:
[cvals2,wvals2,W2]=read_instance()
n2=len(cvals2)

In [9]:
start1=time.time()
all_layers=Algorithm_1(n2,cvals2,wvals2,W2,d)
end1=time.time()
reduced_layers=Algorithm_2(all_layers,n2)
end2=time.time()

In [10]:
#Just printing stuff, no biggie
    
print("Algo 1 Runtime: {}ms".format(1000*(end1-start1)))
print("Algo 2 Runtime: {}ms".format(1000*(end2-end1)))
print("Total Runtime: {}ms".format(1000*(end2-start1)))

Algo 1 Runtime: 255.84936141967773ms
Algo 2 Runtime: 56.98347091674805ms
Total Runtime: 312.8328323364258ms


In [11]:
#Printing node and layer information
for key, layer in all_layers.items():
    print("Layer no: {}".format(key))
    print("Nodes in layer: {}".format(len(layer.values())))
    print("")
    '''for currWeight, nodeInstance in layer.items():
        print("Node: {}".format(currWeight))
        for goingTo, arcCost in nodeInstance.outbound.items():
            print("Arc cost: {}".format(arcCost))
    print("")'''

Layer no: 0
Nodes in layer: 1

Layer no: 1
Nodes in layer: 2

Layer no: 2
Nodes in layer: 4

Layer no: 3
Nodes in layer: 6

Layer no: 4
Nodes in layer: 12

Layer no: 5
Nodes in layer: 21

Layer no: 6
Nodes in layer: 38

Layer no: 7
Nodes in layer: 57

Layer no: 8
Nodes in layer: 76

Layer no: 9
Nodes in layer: 93

Layer no: 10
Nodes in layer: 114

Layer no: 11
Nodes in layer: 133

Layer no: 12
Nodes in layer: 150

Layer no: 13
Nodes in layer: 170

Layer no: 14
Nodes in layer: 178

Layer no: 15
Nodes in layer: 199

Layer no: 16
Nodes in layer: 205

Layer no: 17
Nodes in layer: 219

Layer no: 18
Nodes in layer: 230

Layer no: 19
Nodes in layer: 243

Layer no: 20
Nodes in layer: 265

Layer no: 21
Nodes in layer: 282

Layer no: 22
Nodes in layer: 295

Layer no: 23
Nodes in layer: 314

Layer no: 24
Nodes in layer: 323

Layer no: 25
Nodes in layer: 325

Layer no: 26
Nodes in layer: 337

Layer no: 27
Nodes in layer: 352

Layer no: 28
Nodes in layer: 363

Layer no: 29
Nodes in layer: 369

Laye